In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from spqr.evalutils import evaluate_perplexity
import torch

model_path = '/raid/LLM/opt-6.7b/'
device = 'cuda:1'
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.half)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/jwjeong/miniconda3/envs/spqr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [2]:
linear_list = [
    'self_attn.k_proj',
    'self_attn.q_proj',
    'self_attn.v_proj',
    'self_attn.out_proj',
    'fc1',
    'fc2',
]

layers = model.model.decoder.layers

In [ ]:
from spqr.quant_groups import Quantizer, quantize, dequantize, quantize_dequantize
from spqr.spqr_engine import calculate_bit_error_injection_mask_quantized
import time

qweight_dict = {}
row_perm_dict = {}
scale_dict = {}
zero_dict = {}
for i in range(len(layers)):
    qweight_dict[i] = {}
    row_perm_dict[i] = {}
    scale_dict[i] = {}
    zero_dict[i] = {}
    layer = layers[i]
    sublayers = {name: sublayer for name, sublayer in layer.named_modules() if name in linear_list}
    for name, sublayer in sublayers.items():
        tick = time.time()
        weight = sublayer.weight.detach().clone()
        col_perm, dead, H_inv_diag = torch.load(f'collected_H/opt_6.7b_seed0_H_info/layers.{i}.{name}.pt')
        weight = weight[:, col_perm]
        weight[:, dead] = 0
        weight = weight.to(device)
        out_dim, in_dim = weight.shape

        quantizer = Quantizer(weight.shape)
        quantizer.configure(4, True, False)
        quantizer.find_params(weight, weight=True)
        row_perm = torch.argsort(quantizer.scale.T.squeeze(), descending=True).to(device)
        qweight = quantize(weight, quantizer.scale, quantizer.zero, quantizer.maxq)
        qweight = qweight[row_perm, :]
        qweight_dict[i][name] = qweight.to(torch.int8)
        row_perm_dict[i][name] = row_perm
        scale_dict[i][name] = quantizer.scale
        zero_dict[i][name] = quantizer.zero

elapsed time - layer0 self_attn.k_proj quant. = 0.020861148834228516
elapsed time - layer0 self_attn.v_proj quant. = 0.01961207389831543
elapsed time - layer0 self_attn.q_proj quant. = 0.019977092742919922
elapsed time - layer0 self_attn.out_proj quant. = 0.01851677894592285
elapsed time - layer0 fc1 quant. = 0.12583065032958984
elapsed time - layer0 fc2 quant. = 0.13869667053222656
elapsed time - layer1 self_attn.k_proj quant. = 0.03807497024536133
elapsed time - layer1 self_attn.v_proj quant. = 0.03750133514404297
elapsed time - layer1 self_attn.q_proj quant. = 0.03751397132873535
elapsed time - layer1 self_attn.out_proj quant. = 0.038033246994018555
elapsed time - layer1 fc1 quant. = 0.1351490020751953
elapsed time - layer1 fc2 quant. = 0.13621830940246582
elapsed time - layer2 self_attn.k_proj quant. = 0.03778243064880371
elapsed time - layer2 self_attn.v_proj quant. = 0.046547889709472656
elapsed time - layer2 self_attn.q_proj quant. = 0.038790225982666016
elapsed time - layer2 se

In [ ]:
seed = 0
for i in range(len(layers)):
    layer = layers[i]
    sublayers = {name: sublayer for name, sublayer in layer.named_modules() if name in linear_list}
    for name, sublayer in sublayers.items():
        qweight = qweight_dict[i][name]
        row_perm = row_perm_dict[i][name]
        col_perm, _, _ = torch.load(f'collected_H/opt_6.7b_seed0_H_info/layers.{i}.{name}.pt')
        err_matrix = calculate_bit_error_injection_mask_quantized(
            qweight, 1e-3, seed, 4, 100
        ).reshape_as(qweight)
        err_mask_row = round(qweight.shape[0] * (1/100))
        err_mask_col = round(qweight.shape[1] * (1/100))
        err_matrix[:err_mask_row, :err_mask_col] = 0
        qweight = qweight.to(torch.int32) ^ err_matrix.to(device)
        row_invperm = torch.argsort(row_perm)
        qweight = qweight[row_invperm, :]
        scale = scale_dict[i][name]
        zero = zero_dict[i][name]
        dqweight = dequantize(qweight, scale, zero)

        col_invperm = torch.argsort(col_perm)
        dqweight = dqweight[:, col_invperm]
        sublayer.weight.data = dqweight.to(device)
        seed = seed + 10


KeyboardInterrupt: 

In [19]:
if model.device == device:
    print(evaluate_perplexity(model, tokenizer))
else:
    print(evaluate_perplexity(model.to(device), tokenizer))

Perplexity 7077.686: 100%|██████████| 140/140 [00:32<00:00,  4.35it/s]

7077.685546875
